In [1]:
import pandas as pd
from sklearn.ensemble import IsolationForest

In [2]:
#bases de dados
df_treinamento = pd.read_csv('treinamento.csv', delimiter=';', low_memory=False)
df_treinamento['ANSWER'] = pd.to_numeric(df_treinamento['ANSWER'], errors='coerce')
df_treinamento = df_treinamento.dropna(subset=['ANSWER'])

#limpeza da base de dados, deixando apenas as respostas em número.
df_teste = pd.read_csv('teste.csv', delimiter=';', low_memory=False)
df_teste['ANSWER'] = pd.to_numeric(df_teste['ANSWER'], errors='coerce')
df_teste = df_teste.dropna(subset=['ANSWER'])

In [3]:
#dicionário com os outliers
outliers_dict = {}

In [4]:
for loja in df_treinamento['CODE'].unique():
    #separando a base de treinamento e teste por lojas
    treino_loja = df_treinamento[df_treinamento['CODE'] == loja]
    teste_loja = df_teste[df_teste['CODE'] == loja].copy()
    
    if not treino_loja.empty and not teste_loja.empty:
        #modelo de isolation forest
        model = IsolationForest(contamination=0.1)  # Ajuste a contaminação conforme necessário
        
        #treinando o modelo
        model.fit(treino_loja[['ANSWER']])
        
        #prevendo as respostas para tentar identificar outlier
        teste_loja.loc[:, 'outlier'] = model.predict(teste_loja[['ANSWER']])
        
        #armazenar os outliers (previsões = -1 são outliers)
        outliers_dict[loja] = teste_loja[teste_loja['outlier'] == -1]

In [5]:
#printando lojas e outliers
for loja, outliers in outliers_dict.items():
    print(f"Outliers na loja {loja}:")
    print(outliers[['ANSWER']])
    if 'OT_ID' in outliers.columns:
        print(outliers[['OT_ID']]) 

Outliers na loja 0014W00002E9JRLQAM:
       ANSWER
25919  4160.0
25937  5330.0
25943  6370.0
25992  4680.0
26041  2533.0
           OT_ID
25919  375431251
25937  375431269
25943  375431277
25992  375431331
26041  375431388
Outliers na loja 0014W00002E9IRLQAU:
       ANSWER
24147  3960.0
24169   360.0
24179   237.0
24213  1800.0
24230   474.0
24252   664.0
           OT_ID
24147  375416809
24169  375416831
24179  375416841
24213  375416913
24230  375416935
24252  375416963
Outliers na loja 0014W00002E9KGYQAE:
       ANSWER
37601  4680.0
37615   264.0
37624   275.0
37630   976.0
37658   732.0
37665   504.0
           OT_ID
37601  375504055
37615  375504069
37624  375504078
37630  375504084
37658  375504118
37665  375504126
Outliers na loja 0018A00002WCORKAAV:
     ANSWER
859  1892.0
888  1675.0
891   258.0
894   430.0
933  1032.0
948   430.0
950   240.0
966   439.0
         OT_ID
859  375137823
888  375137869
891  375137877
894  375137880
933  375137921
948  375137938
950  375137940
966 